In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
movie_data = pd.read_json('../data/movies.json')
movie_data = movie_data[movie_data['Budget'].isin([i for i in movie_data['Budget'] if "$"in i])]

In [3]:
def bs(x):
    return BeautifulSoup(x, 'lxml').get_text()

trans_table = ''.join( [chr(i) for i in range(128)] + ['?'] * 128 )
def ascii(s):
    if isinstance(s, unicode):
        return s.encode('ascii', 'replace')
    else:
        return s.translate(trans_table)

In [4]:
movie_data['Director 1'] = [ascii(i[0]) for i in movie_data['Director']]
movie_data['Director 2'] = [ascii(i[1]) if len(i) ==2 else "" for i in movie_data['Director']]

movie_data['Writer 1'] = [ascii(i[0]) for i in movie_data['Writers']]
movie_data['Writer 2'] = [ascii(i[1]) if len(i) >=2 else "" for i in movie_data['Writers']]
movie_data['Writer 2'] = [ascii(i[2]) if len(i) ==3 else "" for i in movie_data['Writers']]

movie_data['Star 1'] = [ascii(i[0]) if len(i) >=1 else "" for i in movie_data['Writers']]
movie_data['Star 2'] = [ascii(i[1]) if len(i) >=2 else "" for i in movie_data['Writers']]
movie_data['Star 3'] = [ascii(i[2]) if len(i) ==3 else "" for i in movie_data['Writers']]

movie_data['Reviews'] = movie_data['Reviews'].str.replace(',','')
movie_data['Score'] = movie_data['Score'].apply(pd.to_numeric)
movie_data['Reviews'] = movie_data['Reviews'].apply(pd.to_numeric)
movie_data['Genres2'] = [','.join([ascii(j) for j in i]) for i  in movie_data['Genres']]
movie_data['Directors'] = [','.join([ascii(j) for j in i]) for i  in movie_data['Director']]
movie_data['Stars'] = [','.join([ascii(j) for j in i]) for i  in movie_data['Stars']]
movie_data['Writers'] = [','.join([ascii(j) for j in i]) for i  in movie_data['Writers']]
movie_data['Storyline'] = [bs(i) for i in movie_data['Storyline']]
movie_data['Plot_Summary'] = [bs(i) for i in movie_data['Plot_Summary']]
movie_data['Budget($)'] = [i.replace(' ','').replace(",",'').replace('$','').split('(')[0] 
                           for i in movie_data['Budget']]
movie_data['Budget($)'] = movie_data['Budget($)'].apply(pd.to_numeric)

In [5]:
gl = []
for i in movie_data['Genres']:
    for j in i:
        gl.append(j.replace(' ',''))
gl = list(set(gl))

for g in gl:
    movie_data[g] = [1 if g in i else 0 for i in movie_data['Genres2']]
    
del movie_data['Genres']
del movie_data['Budget']
del movie_data['Director']

In [9]:
movie_data.to_json('./data/movies_cleaned.json')